In [31]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier


In [33]:
df=pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])
df.dropna(inplace=True)


In [34]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)
x_train.head()

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542
253,30.0,16.1000
719,33.0,7.7750
666,25.0,13.0000


In [49]:
kbins_age=KBinsDiscretizer(n_bins=15 , strategy='kmeans',encode='ordinal')
kbins_fare=KBinsDiscretizer(n_bins=15 , strategy='kmeans',encode='ordinal')

In [50]:
ct=ColumnTransformer([
  ('age_transform',kbins_age,[0]),
  ('fare_transform',kbins_fare,[1])
])
x_train_trf= ct.fit_transform(x_train)
x_test_trf=ct.transform(x_test)

In [51]:
output=pd.DataFrame({'Age':x_train['Age'],'age_trf':x_train_trf[:,0],'Fare':x_train['Fare'],'Fare_trf':x_train_trf[:,1]})
output['age_labels'] = pd.cut(x=x_train['Age'],
                                    bins=ct.named_transformers_['age_transform'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=x_train['Fare'],
                                    bins=ct.named_transformers_['fare_transform'].bin_edges_[0].tolist())
output.sample(5)

,Age,age_trf,Fare,Fare_trf,age_labels,fare_labels
628,26.0,4.0,7.8958,0.0,"(21.557, 26.648]","(0.0, 11.241]"
188,40.0,7.0,15.5000,1.0,"(37.419, 42.902]","(11.241, 19.869]"
269,35.0,6.0,135.6333,10.0,"(32.145, 37.419]","(123.564, 144.756]"
133,29.0,5.0,26.0000,2.0,"(26.648, 32.145]","(19.869, 28.065]"
336,29.0,5.0,66.6000,6.0,"(26.648, 32.145]","(59.337, 72.273]"


In [52]:
clf=DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
ypred=clf.predict(x_test_trf)
accuracy_score(y_test,ypred)


0.6433566433566433

In [61]:
#Binarization

df1=pd.read_csv('train.csv')
df1['family']=df1['SibSp'] + df1['Parch']
df1.drop(columns=['SibSp','Parch'],inplace=True)
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,0


In [58]:
from sklearn.preprocessing import Binarizer
Bin=Binarizer()